# Νευρωνικά Δίκτυα και Βαθιά Μάθηση

## Φώτιος Κούτσικος



# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. **Pre-training σε μεγάλα unlabelled datasets**:

  Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. **Fine-tuning σε μικρότερα labelled datasets**:
  
     Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του **text-classification pipeline** μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης.

Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες: **[entailment/neutral/contradiction]**.

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι **γραμματικά ορθό (acceptable) ή όχι (unacceptable)**:

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια.
Το  Yelp κατασκευάστηκε θεωρώντας τα αστέρια 1 και 2 αρνητικά και τα 3 και 4 θετικά.  Η αρνητική πολικότητα ανήκει στην κατηγορία 1 και η θετική στην κατηγορία 2. Τα reviews αυτά χωρίζονται σε αυτές τις κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.



In [ ]:
from datasets import load_dataset, concatenate_datasets
import numpy as np

# insert your code here
dataset = load_dataset("yelp_polarity")

README.md:   0%|          | 0.00/8.93k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνεται να διατηρούμε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγχουμε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρούμε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [ ]:
dataset["train"][0]

{'text': "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars.",
 'label': 0}

In [ ]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]


print("The number of positive reviews in the Train Dataset is:", train_dataset['label'].count(0))
print("The number of negative reviews in the Train Dataset is:", train_dataset['label'].count(1))
print()
print("The number of positive reviews in the Test Dataset is:", test_dataset['label'].count(0))
print("The number of negative reviews in the Test Dataset is:", test_dataset['label'].count(1))
print()

# We are keeping 300 for each dataset, 150 positive and 150 negative
train_remove_pos = 280000 - 150
train_remove_neg = 280000 - 150

train_pos_ds = train_dataset.filter(lambda example: example['label'] == 1) # Filter for positive reviews
train_neg_ds = train_dataset.filter(lambda example: example['label'] == 0) # Filter for negative reviews


train_pos_drop_indices = np.random.choice(train_pos_ds.num_rows, train_remove_pos, replace=False)
train_neg_drop_indices = np.random.choice(train_neg_ds.num_rows, train_remove_neg, replace=False)

train_pos_ds = train_pos_ds.select(np.delete(np.arange(train_pos_ds.num_rows),train_pos_drop_indices)) # Select the remaining positive reviews
train_neg_ds = train_neg_ds.select(np.delete(np.arange(train_neg_ds.num_rows),train_neg_drop_indices)) # Select the remaining negative reviews

train_dataset = concatenate_datasets([train_pos_ds, train_neg_ds]) # Concatenate the remaining positive and negative reviews

# We are doing the same for the test dataset
test_remove_pos = 19000 - 150
test_remove_neg = 19000 - 150

test_pos_ds = test_dataset.filter(lambda example: example['label'] == 1) # Filter for positive reviews
test_neg_ds = test_dataset.filter(lambda example: example['label'] == 0) # Filter for negative reviews


test_pos_drop_indices = np.random.choice(test_pos_ds.num_rows, test_remove_pos, replace=False)
test_neg_drop_indices = np.random.choice(test_neg_ds.num_rows, test_remove_neg, replace=False)

test_pos_ds = test_pos_ds.select(np.delete(np.arange(test_pos_ds.num_rows),test_pos_drop_indices)) # Select the remaining positive reviews
test_neg_ds = test_neg_ds.select(np.delete(np.arange(test_neg_ds.num_rows),test_neg_drop_indices)) # Select the remaining negative reviews

test_dataset = concatenate_datasets([test_pos_ds, test_neg_ds]) # Concatenate the remaining positive and negative reviews

print()
print("The number of positive reviews in the Train Dataset after preprocessing is:", train_dataset['label'].count(0))
print("The number of negative reviews in the Train Dataset after preprocessing is:", train_dataset['label'].count(1))
print()
print("The number of positive reviews in the Test Dataset after preprocessing is:", test_dataset['label'].count(0))
print("The number of negative reviews in the Test Dataset after preprocessing is:", test_dataset['label'].count(1))



The number of positive reviews in the Train Dataset is: 280000
The number of negative reviews in the Train Dataset is: 280000

The number of positive reviews in the Test Dataset is: 19000
The number of negative reviews in the Test Dataset is: 19000



Filter:   0%|          | 0/560000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/560000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/38000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/38000 [00:00<?, ? examples/s]


The number of positive reviews in the Train Dataset after preprocessing is: 150
The number of negative reviews in the Train Dataset after preprocessing is: 150

The number of positive reviews in the Test Dataset after preprocessing is: 150
The number of negative reviews in the Test Dataset after preprocessing is: 150


In [ ]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 300
})

In [ ]:
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 300
})

# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα.

Η διαδικασία αυτή επιτελείται μέσω των **Tokenizers**, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνεται να ακολουθήσουμε στη συνέχεια.

Συγκεκριμένα, **αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer**. Επιλέγοντας τη μέθοδο **from_pretrained** λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλούμαστε να επιτελέσουμε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής μας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξουμε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώνουμε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'distilbert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Η συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξαμε. Το εφαρμόζουμε τόσο στο train, όσο και στο test set.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_train_dataset
test_dataset = tokenized_test_dataset


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Τυπώνοντας το train ή το test set, βλέπουμε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθήκαμε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

In [ ]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 300
})

In [ ]:
test_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 300
})

## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Θα τη χρησιμοποιήσοθμε ως εναλλακτική του να γράψουμε το δικό μας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [ ]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορούμε να πειραματιστούμε κατά τη διαδικασία fine-tuning.


Καλούμαστε να πειραματιστούμε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσουμε optimizer και scheduler για το fine-tuning. Θα εκτελέσουμε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Δίνονται σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσαμε και το accuracy που πετύχαμε στην τελευταία εποχή.

2. Βάσει των πειραματισμών, σχολιάζουμε και αναλύουμε πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξαμε.

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="test_trainer",
    eval_strategy="epoch",
    per_device_train_batch_size=16)


# optimizer
args = args.set_optimizer('adamw_torch')
# scheduler
args = args.set_lr_scheduler()
# learning rate and batch size
args = args.set_training(learning_rate=1e-5, batch_size=8, num_epochs=3)
args1 = args.set_training(learning_rate=1e-4, batch_size=8, num_epochs=3)
args2 = args.set_training(learning_rate=1e-5, batch_size=16, num_epochs=3)
args3 = args.set_training(learning_rate=1e-4, batch_size=16, num_epochs=3)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

model1 = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer1 = Trainer(
    model=model1,
    args=args1,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

model2 = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer2 = Trainer(
    model=model2,
    args=args2,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

model3 = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer3 = Trainer(
    model=model3,
    args=args3,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-strea

Στη συνέχεια, ρυθμίζουμε (fine-tune) το μοντέλο σας καλώντας το [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
trained_model=trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.572438,0.723333
2,No log,0.255562,0.893333
3,No log,0.344317,0.873333


In [ ]:
trained_model=trainer1.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.444531,0.800000
2,No log,0.371533,0.856667
3,No log,0.283467,0.906667


In [ ]:
trained_model=trainer2.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.444531,0.800000
2,No log,0.371533,0.856667
3,No log,0.283467,0.906667


In [ ]:
trained_model=trainer3.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.372972,0.863333
2,No log,0.283571,0.886667
3,No log,0.352167,0.873333


α. Οι υπερπαράμετροι που χρησιμοποιήσαμε σε κάθε πείραμα και το accuracy που σημείωσε το καθένα στη τελευταία εποχή:

| Arguments | Learning Rate | Batch Size | Accuracy |
|-----------|---------------|------------|----------|
| args      | 1e-5          | 8          | 87.33%   |
| args1     | 1e-4          | 8          | 90.67%   |
| args2     | 1e-5          | 16         | 90.67%   |
| args3     | 1e-4          | 16         | 87.33%   |

β. Τα πειραματικά αποτελέσματα δείχνουν ότι τόσο ο συνδυασμός μικρού batch size με μεγαλύτερο learning rate (8, 1e-4), όσο και ο συνδυασμός μεγαλύτερου batch size με μικρότερο learning rate (16, 1e-5) οδηγούν σε καλύτερη απόδοση, επιτυγχάνοντας 90.67% accuracy. Αντίθετα, όταν χρησιμοποιούνται είτε και τα δύο μικρά είτε και τα δύο μεγάλα, η ακρίβεια μειώνεται (~87.33%). Αυτό υποδεικνύει ότι πρέπει να υπάρχει ισορροπία μεταξύ learning rate και batch size, προκειμένου να ενισχυθεί η αποτελεσματικότητα της εκπαίδευσης.

# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε θα πραγματοποιήσουμε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή λογική (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέγουμε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [ ]:
# load dataset
from datasets import load_dataset

dataset = load_dataset("piqa")

val_ds = dataset["validation"]
val_ds = val_ds.select(range(100))

val_ds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.41k [00:00<?, ?B/s]

piqa.py:   0%|          | 0.00/5.36k [00:00<?, ?B/s]

The repository for piqa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/piqa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/16113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3084 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

Dataset({
    features: ['goal', 'sol1', 'sol2', 'label'],
    num_rows: 100
})

Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Θα καταγράψουμε λοιπόν το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης θα χρησιμοποιήσουμε τουλάχιστον 5 κατάλληλα μοντέλα.

In [ ]:
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
# models
model_names = [
    "sileod/deberta-v3-base-tasksource-nli",
    "facebook/bart-large-mnli",
    "joeddav/xlm-roberta-large-xnli",
    "MoritzLaurer/deberta-v3-large-zeroshot-v2.0",
    "typeform/distilbert-base-uncased-mnli"
]

In [ ]:
# function for ending prediction
from transformers import pipeline

for name in model_names:
  classifier = pipeline("zero-shot-classification",model=name)

  correct = 0
  for idx in range(100):
    text = val_ds['goal'][idx]
    candidate_labels = [val_ds['sol1'][idx], val_ds['sol2'][idx]]

    prediction = classifier(text, candidate_labels)

    best_label = prediction['labels'][0]

    if best_label == candidate_labels[0]:
      result = 0
    else: result = 1

    if val_ds['label'][idx] == result:
      correct += 1

  accuracy = correct / 100
  print("Accuracy of ", name, "is: ", accuracy*100, "%.")

Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Accuracy of  sileod/deberta-v3-base-tasksource-nli is:  71.0 %.


Device set to use cuda:0


Accuracy of  facebook/bart-large-mnli is:  60.0 %.


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


Accuracy of  joeddav/xlm-roberta-large-xnli is:  49.0 %.


Device set to use cuda:0
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Accuracy of  MoritzLaurer/deberta-v3-large-zeroshot-v2.0 is:  74.0 %.


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Device set to use cuda:0


Accuracy of  typeform/distilbert-base-uncased-mnli is:  53.0 %.


## B2. Truthful QA

### Sentence Transformers

Οι **sentence transformers** χρησιμοποιούνται για να δημιουργήσουν **embeddings προτάσεων**, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελώντας το ακόλουθο κελί, δίνεται μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). 

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

np.float32(0.40488452)

Για τη συνέχεια της άσκησης, καλούμαστε να επιλέξουμε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή.

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορούμε να θεωρήσουμε τα ίδια μοντέλα και να τα επεκτείνουμε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Θα γράψουμε λοιπόν μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξαμε στο ερώτημα Β1.


In [ ]:
# load dataset
from datasets import load_dataset, Dataset

dataset = load_dataset("truthful_qa", "generation")
ds = dataset["validation"]

similarity_ds = []
for example in ds:
     if len(example["correct_answers"]) >= 2 and len(example["incorrect_answers"]) >= 1:
        similarity_ds.append({
            "question": example["question"],
            "best": example["best_answer"],
            "correct1": example["correct_answers"][0],
            "correct2": example["correct_answers"][1],
            "incorrect": example["incorrect_answers"][0]
        })

similarity_ds = similarity_ds[:100]
similarity_ds = Dataset.from_list(similarity_ds)
similarity_ds

README.md:   0%|          | 0.00/9.59k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/223k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'best', 'correct1', 'correct2', 'incorrect'],
    num_rows: 100
})

In [ ]:
# load models for semantic similarity and QA
similarity_models = [
    'sentence-transformers/all-MiniLM-L6-v2',
    'sentence-transformers/all-mpnet-base-v2',
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    'sentence-transformers/use-cmlm-multilingual',
    'sentence-transformers/paraphrase-MiniLM-L6-v2',
    'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
]

In [ ]:
# function for optimal correct answers & semantic similarity
from transformers import pipeline

classifier = pipeline("zero-shot-classification",model="facebook/bart-large-mnli", hypothesis_template="{}")

for smodel in similarity_models:
    correct = 0

    model = SentenceTransformer(smodel)


    for idx in range(100):
        sentence = similarity_ds['question'][idx]
        sentence1 = similarity_ds['correct1'][idx]
        sentence2 = similarity_ds['correct2'][idx]
        best = similarity_ds['best'][idx]
        incorrect = similarity_ds['incorrect'][idx]

        candidate_labels = [best, sentence1, sentence2, incorrect]

        prediction = classifier(sentence, candidate_labels)
        selected = prediction['labels'][0]
        embeddings = model.encode([best, selected], convert_to_tensor=True)


        sim = get_cosine_similarity(embeddings[0], embeddings[1])

        if selected in (sentence1, sentence2) and sim > 0.95:
            correct += 1

    accuracy = correct / 100
    print("Accuracy of ", model, "is: ", accuracy * 100, "%.")

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Accuracy of  SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
) is:  31.0 %.


## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.

Για λόγους επιτάχυνσης, επιλέγουμε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [ ]:
# load dataset
from datasets import load_dataset

dataset = load_dataset("winogrande", 'winogrande_xs')
train_ds = dataset["train"]
train_ds = train_ds.select(range(100))

train_ds

Generating train split:   0%|          | 0/160 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1767 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1267 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'option1', 'option2', 'answer'],
    num_rows: 100
})

Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), θα καταγράψουμε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να ανάγουμε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάζουμε  3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίζουμε το πρόβλημα του Winogrande.

In [ ]:
# load models
models = [
    "bert-base-uncased",
    "albert-base-v2",
    "roberta-base"
]

In [ ]:
# create pipelines
from transformers import pipeline

pipelines = {}
for model in models:
  pipelines[model] = pipeline("fill-mask", model=model)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# function for predicting best fill

for model in models:
  total_considered = 0
  correct = 0

  mask = pipelines[model].tokenizer.mask_token
  for idx in range(100):
    sentence = train_ds['sentence'][idx]

    masked_sentence = sentence.replace("_", mask)

    predictions = pipelines[model](masked_sentence)
    top_preds = [p["token_str"].strip().lower() for p in predictions[:5]]

    if (train_ds['option1'][idx].lower() not in top_preds) and (train_ds['option2'][idx].lower() not in top_preds):
      continue
    else:
      if (train_ds['option1'][idx].lower() not in top_preds):
        total_considered += 1
        op1_score = 0
        op2_index = top_preds.index(train_ds['option2'][idx].lower())
        op2_score = predictions[op2_index]['score']
      elif (train_ds['option2'][idx].lower() not in top_preds):
        total_considered += 1
        op2_score = 0
        op1_index = top_preds.index(train_ds['option1'][idx].lower())
        op1_score = predictions[op1_index]['score']
      else:
        total_considered += 1
        op1_index = top_preds.index(train_ds['option1'][idx].lower())
        op2_index = top_preds.index(train_ds['option2'][idx].lower())

        op1_score = predictions[op1_index]['score']
        op2_score = predictions[op2_index]['score']

        predictions[op1_index]['token_str']
        predictions[op2_index]['token_str']

    if op1_score > op2_score:
      pred = 1
    else: pred = 2

    ans = int(train_ds['answer'][idx])

    if pred == ans:
      correct += 1

  accuracy = correct / total_considered
  print("Accuracy of ", model, "is: ", accuracy * 100, "%.")


Accuracy of  bert-base-uncased is:  56.17977528089888 %.
Accuracy of  albert-base-v2 is:  53.48837209302325 %.
Accuracy of  roberta-base is:  57.446808510638306 %.
